In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\CNS.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
Init();


In [2]:
BoSSSshell.WorkflowMgm.Init("CNS_AcousticWave1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 0; Grid Count = 0; Path = \\dc3\userspace\sebastian\cluster\CNS_AcousticWave1D }

In [3]:
BoSSSshell.GetDefaultQueue()

RuntimeLocation,win\amd64
AdditionalEnvironmentVars,"[ [OMP_PROC_BIND, spread] ]"
DeploymentBaseDirectory,\\dc3\userspace\sebastian\cluster\binaries
DeployRuntime,True
Name,Default
DotnetRuntime,dotnet
Username,FDY\sebastian
NumOfAdditionalServiceCores,0
NumOfAdditionalServiceCoresMPISerial,0
NumOfServiceCoresPerMPIprocess,0
ServerName,DC3


In [4]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\CNS_AcousticWave1D

In [5]:
double[] PerStartTimes = new double[] {10};
int[] Configs = new int[] {1,2,3};
int[] DGdegrees = new int[] {3};
int[] Grid_Res = new int[] {1,2};
double[] Amplitudes = new double[] {1e-5};
string[] WaveForms= new string[] {"bump","1sinus"};
string[] LevelSetConfigs = new string[] {"true"};//,"false"};
//double[] MachLs = GenericBlas.Linspace(2.0,8.0,7);
double[] MachLs = new double[] {1.5};
double[] WLs = new double[] {0.8};
//double[] Aggs = new double[] {0.2,0.4,0.6};
//var gMins = new double[] {1,0.1};
// int[] LSDGdegrees = new int[] { 0};
// int[] Grid_Res = new int[] { 4 , 8  };
// double[] shock_angles = new double[] { 33};

In [6]:
      int savePeriod=1500;
    double endTime=14.0;
    double shockPos=1.5;
    double wavePos=-0.4; 
    string isRestart="false";
            foreach(double waveLength in WLs ){
        foreach(double MachL in MachLs ){
    foreach(double amp in Amplitudes ){
    double amp_neg=0.0;
    double amp_pos=0.0;
foreach(double config in Configs) {
    foreach(string waveform in WaveForms) {
foreach(double perStartTime in PerStartTimes) {
        if(config ==1){ //upstream positive acoustic wave
            amp_neg= 0.0;            amp_pos= amp;            endTime=perStartTime+5;            shockPos=1.5;            wavePos=-0.4;
        }else if(config==2){ //downstream negative acoustic wave
            amp_neg= amp;            amp_pos= 0.0;            endTime=perStartTime+20;            shockPos=0.5;            wavePos=3.2;
        }
        else if(config==3){ //upstream positive acoustic wave
            amp_neg= amp;            amp_pos= 0.0;            endTime=perStartTime+10;            shockPos=1.5;            wavePos=-0.4;
        }
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
                //int cells=(n+1)*30;
                int xCells =(n+1)*30+1;
                int yCells =3;
                Job j = new Job(string.Format($"CNSAcousticWave1D_p{p}_xCells{xCells}_{waveform}_Mach{MachL}_wP{wavePos}_sP{shockPos}_amp_neg{amp_neg}_amp_pos{amp_pos}_pST{perStartTime}_wL{waveLength}_rc0"),typeof(CNS.CNSProgram));
                // job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                
                j.RetryCount = 2;
                j.NumberOfThreads=1;
                j.MySetCommandLineArguments("--control", $"cs:CNS.ControlExamples_Supersonic.AcousticWave( isRestart:{isRestart},waveLength:{waveLength},savePeriod:{savePeriod},perStartTime:{perStartTime},endTime:{endTime},MachL:{MachL},wavePosition:{wavePos}, shockPosition:{shockPos},p_amp_neg: {amp_neg}, p_amp_pos:{amp_pos},waveform:\"{waveform}\",dgDegree:{p},numOfCellsX:{xCells},numOfCellsY:{yCells},dbPath:@\"{dbPath}\")");
                j.Activate(BoSSSshell.ExecutionQueues[0]); 
                
            
        };                                                                                                                                                          
    };
};
    };
}
    };
        }
            }


